In [1]:
import nltk
import re, string
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk import FreqDist
import random
from nltk import classify
from nltk import NaiveBayesClassifier

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
stop_words = stopwords.words('english')

In [2]:
from pandas import DataFrame

In [7]:
df1 = DataFrame (positive_tweets,columns=['tweets'])

In [13]:
df2 =DataFrame (negative_tweets,columns=['tweets'])

In [14]:
df1.shape

(5000, 1)

In [15]:
df1.to_csv('df1.csv')

In [16]:
df2.to_csv('df2.csv')

In [3]:
import pandas as pd

In [76]:
df_p=pd.read_csv('df1.csv')

In [77]:
df_n=pd.read_csv('df2.csv')

In [78]:
df=pd.concat([df_p,df_n])
#we add the sentiment column after download it

In [4]:
df=pd.read_csv('df.csv')

In [5]:
df=df.dropna()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 10001
Data columns (total 4 columns):
Unnamed: 0      10000 non-null int64
Unnamed: 0.1    10000 non-null object
tweets          10000 non-null object
sentiment       10000 non-null object
dtypes: int64(1), object(3)
memory usage: 390.6+ KB


In [7]:
def remove_URL(headline_text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', headline_text)

In [8]:
df['tweets']=df['tweets'].apply(remove_URL)

In [9]:
def remove_html(headline_text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',headline_text)

df['tweets']=df['tweets'].apply(remove_html)


In [10]:
import re, string

In [11]:
def remove_punct(headline_text):
    table=str.maketrans('','',string.punctuation)
    return headline_text.translate(table)
df['tweets']=df['tweets'].apply(remove_punct)

In [12]:
df.head(3)

,Unnamed: 0,Unnamed: 0.1,tweets,sentiment
0,0,0,FollowFriday FranceInte PKuchly57 MilipolParis...,Positive
1,1,1,Lamb2ja Hey James How odd Please call our Con...,Positive
2,2,2,DespiteOfficial we had a listen last night As...,Positive


In [13]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return df['tweets']==df1['tweets'].apply(remove_stopwords)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import nltk
normalization = None
normalization = 'stemmer'
normalization = 'lemmatizer'
def stem_tokens(tokens):
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens
def lemmatize_tokens(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens
def normalize_tokens(normalization):
    if normalization is not None:
        if normalization == 'stemmer':
            df['tweets']==df['tweets'].apply(stem_tokens)
        elif normalization == 'lemmatizer':
            df['tweets']==df['tweets'].apply(lemmatize_tokens)
        
normalize_tokens(normalization)

In [15]:
df.head(3)

,Unnamed: 0,Unnamed: 0.1,tweets,sentiment
0,0,0,FollowFriday FranceInte PKuchly57 MilipolParis...,Positive
1,1,1,Lamb2ja Hey James How odd Please call our Con...,Positive
2,2,2,DespiteOfficial we had a listen last night As...,Positive


In [16]:
df = df[["tweets", "sentiment"]]
df.head()

,tweets,sentiment
0,FollowFriday FranceInte PKuchly57 MilipolParis...,Positive
1,Lamb2ja Hey James How odd Please call our Con...,Positive
2,DespiteOfficial we had a listen last night As...,Positive
3,97sides CONGRATS,Positive
4,yeaaaah yippppy my accnt verified rqst has su...,Positive


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

In [23]:
X = df["tweets"]
y = df["sentiment"]
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
count_vect = CountVectorizer()
X=vectorizer.fit_transform(X).toarray()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)

#count_vect = CountVectorizer()

#X_train_counts = count_vect.fit_transform(X_train)

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [26]:
predictions = text_classifier.predict(X_test)

In [27]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [28]:
print(accuracy_score(y_test, predictions))

0.761


In [29]:
print(classification_report(y_test,predictions))  

              precision    recall  f1-score   support

    Negative       0.72      0.84      0.78       988
    Positive       0.81      0.68      0.74      1012

    accuracy                           0.76      2000
   macro avg       0.77      0.76      0.76      2000
weighted avg       0.77      0.76      0.76      2000



In [30]:
# pip_install tweepy
# pip_install pandas
# pip_install numpy
# pip install matplotlib
# pip install textblob

from textblob import TextBlob
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Our Twitter API Credentials:
# (Normally, we store these in a seperate file for precautionary measures.
#  However, for the sake of simplicity, we are providing them here.)
Consumer_Key = 'j1PZszWJPTn50m9CyZaKQEL4V'
Consumer_Secret = '0VLZXNHBidBXOYELBSmAJnWpsohTlMQ2flop7cGY4IRd7gRPc8'        
Access_Token = '1334510916344360963-6ESgNupKmKn8yuCEEpi91XLO7cXttC'    
Access_Token_Secret = 'O4ghcRniTGXb9V7LIcfW5CpK93Pc6yfTiRY5vxSZReHuq'

class Twitter_Customer():

    def __init__(self, twitter_user = None):
        self.auth = Authentication().authenticate_twitter_app()
        self.twitter_client = tweepy.API(self.auth)
        self.twitter_user = twitter_user

    def twitter_customer_api(self):
        return self.twitter_client

    
class Authentication():

    def authenticate_twitter_app(self):
        auth = tweepy.OAuthHandler(Consumer_Key, Consumer_Secret)
        auth.set_access_token(Access_Token, Access_Token_Secret)
        return auth

    
class My_Stream_Listener():

    def __init__(self):
        self.twitter_autenticator = Authentication()    

    def stream_tweets(self, fetched_tweets_filename):
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = tweepy.Stream(auth, listener)
        
class TwitterListener(tweepy.StreamListener):

    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            return False
        print(status)
        
class TweetAnalyzer():

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(RT[\s]+)", " ", tweet).split())

    def analyze_sentiment_polarity(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity < 0:
            return 'Negative'
        elif analysis.sentiment.polarity == 0:
            return 'Neutral'
        else:
            return 'Positive'
    
    def analyze_sentiment_subjectivity(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        return round(analysis.sentiment.subjectivity, 2)

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        return df
 
if __name__ == '__main__':

    twitter_client = Twitter_Customer()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.twitter_customer_api()

    #tweets = api.user_timeline(screen_name = "TheRock", count=200)
    tweets = api.search(q = "#Apple",count = 10000, lang = "en")
    
    df_tweet_api = tweet_analyzer.tweets_to_data_frame(tweets)
    df_tweet_api['sentiment'] = np.array([tweet_analyzer.analyze_sentiment_polarity(tweet) for tweet in df_tweet_api['tweets']])
    df_tweet_api['subjectivity'] = np.array([tweet_analyzer.analyze_sentiment_subjectivity(tweet) for tweet in df_tweet_api['tweets']])
    df_tweet_api.to_csv('twitter_data.csv')

df_tweet_api = pd.read_csv('twitter_data.csv')
df_tweet_api.head()

,Unnamed: 0,tweets,date,len,likes,retweets,sentiment,subjectivity
0,0,The latest Leverage Credit Recovery! https://t...,2020-12-14 03:32:50,125,0,0,Positive,0.55
1,1,RT @flaugher: @mbrochh I sent mine back. Too h...,2020-12-14 03:31:00,140,0,1,Neutral,0.27
2,2,@mbrochh I sent mine back. Too heavy for those...,2020-12-14 03:30:44,140,0,1,Neutral,0.27
3,3,RT @bebecitaaa__: new video is LIVE! \n\ndon’t...,2020-12-14 03:30:15,144,0,3,Positive,0.48
4,4,RT @Tree_eeee: i just watch the' night is long...,2020-12-14 03:30:14,140,0,1,Positive,0.65


In [31]:
df_tweet_api=df_tweet_api[df_tweet_api['sentiment']!='Neutral']

In [32]:
df_tweet_api.head()

,Unnamed: 0,tweets,date,len,likes,retweets,sentiment,subjectivity
0,0,The latest Leverage Credit Recovery! https://t...,2020-12-14 03:32:50,125,0,0,Positive,0.55
3,3,RT @bebecitaaa__: new video is LIVE! \n\ndon’t...,2020-12-14 03:30:15,144,0,3,Positive,0.48
4,4,RT @Tree_eeee: i just watch the' night is long...,2020-12-14 03:30:14,140,0,1,Positive,0.65
5,5,RT @shawnmurry: Check out NEW Apple Watch Seri...,2020-12-14 03:30:14,124,0,1,Positive,0.45
7,7,"RT @funtop_india: Among all the FunTop Jams, w...",2020-12-14 03:30:13,140,0,1,Positive,0.50


In [33]:
x2_test=df_tweet_api['tweets']
y2_test=df_tweet_api['sentiment']

In [35]:
X2=vectorizer.transform(x2_test)

In [36]:
predictions =text_classifier.predict(X2)

In [37]:
print(classification_report(y2_test,predictions))  
print(accuracy_score(y2_test, predictions))

              precision    recall  f1-score   support

    Negative       0.74      0.70      0.72        20
    Positive       0.86      0.88      0.87        41

    accuracy                           0.82        61
   macro avg       0.80      0.79      0.79        61
weighted avg       0.82      0.82      0.82        61

0.819672131147541
